# Julia methods as actors

## Installing necessary requirements

1) Install Julia itself. Start from here: <http://julialang.org/downloads/>

2) Install `PyCall` which is a Python binding library for Julia (i.e. calling Python from Julia).

```julia
julia> Pkg.update()
julia> Pkg.add("PyCall")
```

3) Install `pyjulia` Python package:

3a) Download it from github.com: 

```bash
git clone git@github.com:JuliaLang/pyjulia.git
```

3b) Install the package:

```bash
cd pyjulia
pip install .              # Copy to site-packages
# pip install -e .         # Makes link to current directory in site-packages
```

4) Try in Python:

```python
import julia
jl = julia.Julia()        # Takes a few seconds
jl.eval("2 + 2")          # Should immediately return "4"
```

In [1]:
%load_ext autoreload
%autoreload(2)

In [2]:
from wowp.actors.julia import JuliaMethod
from wowp.schedulers import LinearizedScheduler
import numpy as np

## Simple calling

In [3]:
sqrt = JuliaMethod("sqrt", inports="a")
sqrt(4)

2.0

## Calling on numpy arrays

In [4]:
sqrt = JuliaMethod("sqrt", inports="a")

array = np.random.rand(5, 5)
scheduler = LinearizedScheduler()
scheduler.put_value(sqrt.inports.a, array)
scheduler.execute()
sqrt.outports.result.pop()

array([[ 0.89750618,  0.92167126,  0.93271865,  0.61923411,  0.47707956],
       [ 0.89992374,  0.98285476,  0.7510661 ,  0.46864257,  0.89478677],
       [ 0.38486299,  0.95265772,  0.51202745,  0.34914621,  0.16690231],
       [ 0.96218508,  0.83740537,  0.61862326,  0.91165117,  0.85024454],
       [ 0.69284119,  0.6228784 ,  0.44213921,  0.56035767,  0.85448758]])

## Chain sqrt method to pass numpy arrays

In [5]:
sqrt = JuliaMethod("sqrt", inports="a")
sqrt2 = JuliaMethod("sqrt", inports="a")

sqrt.outports.result.connect(sqrt2.inports.a)

array = np.random.rand(5, 5)
scheduler = LinearizedScheduler()
scheduler.put_value(sqrt.inports.a, array)
scheduler.execute()
sqrt2.outports.result.pop()

array([[ 0.59191931,  0.87333554,  0.69372349,  0.83217936,  0.9437529 ],
       [ 0.41729849,  0.91345055,  0.74299889,  0.98324251,  0.8358827 ],
       [ 0.75384593,  0.79854596,  0.8798665 ,  0.6376801 ,  0.61952612],
       [ 0.7640312 ,  0.87039703,  0.78204509,  0.89378438,  0.98533398],
       [ 0.51226547,  0.98575522,  0.51161319,  0.94977028,  0.56050242]])

## Using method from a package

In [6]:
%%file ABCD.jl

module ABCD

VERSION < v"0.4-" && using Docile

export quad

@doc doc"""Fourth power of the argument.""" ->
function quad(a)
    a ^ 4
end

end

Overwriting ABCD.jl


In [7]:
quad = JuliaMethod(package_name="ABCD", method_name="quad", inports="a")
quad(4.0)

256.0

In [8]:
quad.name

'ABCD.quad'

### Non-existent module or package

In [9]:
xxx = JuliaMethod(package_name="ABBD", method_name="x")
xxx()

JuliaError: Exception 'ErrorException' ocurred while calling julia code:
poo

Code:
using ABBD

In [10]:
xxx = JuliaMethod(package_name="ABCD", method_name="xx")
xxx()

JuliaError: Exception 'UndefVarError' ocurred while calling julia code:
poo

Code:
ABCD.xx

### Unicode identifiers

The page of julia states that unicode identifiers are not valid. This is true for automatically imported methods. But not for `JuliaMethod`. Names like `πtimes!` are fine :-)

In [11]:
%%file UnicodePi.jl

module UnicodePi

VERSION < v"0.4-" && using Docile

export πtimes!

@doc doc"""Return pi times argument""" ->
function πtimes!(a)
    π * a
end

end

Overwriting UnicodePi.jl


In [12]:
pi_times = JuliaMethod(package_name="UnicodePi", method_name="πtimes!")
print(pi_times.name)
pi_times(4)

UnicodePi.πtimes!


12.566370614359172